# Dimension Changer

Prerequisites:
- You have preprocessed your adata using some time smoothing method into the data/preprocessed directory. [notebook: 3-time-pseudotime-meld]: 
    - gene values on _X_ (adata.X)
    - phate embeddings of the data on _obsm_ (ex.: adata.obsm['X_phate])
    - phate visualization space of the data on _obsm_ (ex.: adata.obsm['X_phate_visualization])
    - any extra metadata on _obs_ (ex.: adata.obs['time_label'], adata.obs['disease_progression'], etc...)
    - smoothed time in the on _obsm_ (ex.: adata.obsm['smooth_time'])


In this notebook, we will:
- Train a dimension changer network.
# TODO: DEFINE WHAT THE DIM CHANGER IS DOING

In [2]:
import scanpy as sc
import os

PROCESSED_DATA_DIR = os.path.join('../../data', 'processed')
DIMENSION_CHANGER_DIR = os.path.join('../../dimension_changer')
os.makedirs(DIMENSION_CHANGER_DIR, exist_ok=True)

print(PROCESSED_DATA_DIR)
print(DIMENSION_CHANGER_DIR)

../../data/processed
../../dimension_changer


We start by loading the data that has all the necessary variables

In [3]:
adata = sc.read(os.path.join(PROCESSED_DATA_DIR, 'adata_time.h5ad'))
adata

AnnData object with n_obs × n_vars = 17944 × 18019
    obs: 'time_label', 'smooth_time'
    uns: 'pca'
    obsm: 'X_pca', 'X_phate', 'X_phate_visualization'
    varm: 'PCs'

In [4]:
X_phate = adata.obsm['X_phate']
X_pca = adata.obsm['X_pca']
X_phate_vis = adata.obsm['X_phate_visualization']
var_ratio = adata.uns['pca']['variance_ratio']

The dimchanger is currently defined under omics_toolbox/dimchanger.py

In [6]:
from omics_toolbox.dimchanger import DimChanger
dimchanger = DimChanger.train(X_phate, X_pca, X_phate_vis, var_ratio, save_dir=DIMENSION_CHANGER_DIR)

ModuleNotFoundError: No module named 'decoder'